In [1]:
import numpy as np
import matplotlib.pyplot as plt

from data_loader import CIFAR_10_DataLoader
import torch
import losses
from zth_scratchpad.makemore.makemore_batchnorm import n_hidden
%matplotlib inline

In [5]:
file_directory = '/Users/subhojit/Downloads/cifar-10-batches-py'
xtrain_data, ytrain, Xtest, ytest = CIFAR_10_DataLoader().load_cifar_10_dataset(file_directory)

xtrain = xtrain_data.astype(np.float32) / 255.0
Xtest = Xtest.astype(np.float32) / 255.0

n1 = int(0.8 * len(xtrain_data))
Xtrain = xtrain_data[:n1]
ytrain = ytrain[:n1]
Xdev = xtrain_data[n1:]
ydev = ytrain[n1:]

num_classes = len(set(ytrain))

In [8]:
np.random.seed(18)
n_hidden = 200
std_dev = 1e-2
W1 = np.random.randn(3072, n_hidden) * std_dev
b1 = np.zeros(n_hidden)
W2 = np.random.randn(n_hidden, num_classes) * std_dev
b2 = np.zeros(num_classes)
bngain = np.ones((1, n_hidden))
bnbias = np.zeros((1, n_hidden))

parameters = [W1, b1, W2, b2, bngain, bnbias]

print(sum(p.size for p in parameters))

617010


In [9]:
def softmax_loss(scores, y):
    epsilon = 1e-12
    num_examples = scores.shape[0]
    shifted_scores = scores - np.max(scores, axis=1, keepdims=True)
    exp_scores = np.exp(shifted_scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    correct_log_probs = -np.log(probs[np.arange(num_examples), y] + epsilon)
    loss = np.mean(correct_log_probs)
    return loss


def softmax_numpy(x, axis=1):
    x_shifted = x - np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

In [14]:
max_iterations = 100
batch_size = 128
n = batch_size
lossi = []
Hs = []

for i in range(max_iterations):

    #mini batch
    ix = np.random.randint(0, Xtrain.shape[0], (batch_size,))
    Xb, Yb = Xtrain[ix], ytrain[ix]


    #farward pass
    hpreact = np.dot(Xb, W1) + b1
    # batch norm
    bnmeani = (1/n)*hpreact.sum(0, keepdims=True)
    #bnstdi = hpreact.std(0, keepdims=True)
    bndiff = hpreact - bnmeani
    bndiff2 = bndiff**2
    bnvar = 1/(n-1)*(bndiff2).sum(0, keepdims=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    xhat = bndiff * bnvar_inv
    bnout = bngain * xhat + bnbias # element-wise multiply

    h1 = np.maximum(bnout, 0)
    scores = np.dot(h1, W2) + b2

    # backward pass
    loss = softmax_loss(scores, Yb)
    lossi.append(loss)
    dscores = softmax_numpy(scores)
    dscores[np.arange(Xb.shape[0]), Yb] -= 1
    dscores /= Xb.shape[0]

    dW2 = np.dot(h1.T, dscores)
    db2 = np.sum(dscores, axis=0)
    dh1 = np.dot(dscores, W2.T)
    dbnout = dh1 * (bnout > 0)
    dbnbias = dbnout.sum(0, keepdims=True)
    dxhat = bngain * dbnout
    dbngain = (dbnout * xhat).sum(0, keepdims=True)
    dbnvar_inv = (dxhat * bndiff).sum(0, keepdims=True)
    dbndiff = bnvar_inv * dxhat
    dbnvar_inv =


    dhpreact = 1

    dW1 = np.dot(Xb.T, dhpreact)
    db1 = np.sum(dhpreact, axis=0)

    grads = [dW1, db1, dW2, db2]
    # print(grads)
    lr = 0.01
    W1 += -lr * dW1
    b1 += -lr * db1
    W2 += -lr * dW2
    b2 += -lr * db2
    if i % 100 == 0:
        print(f"loss: {loss:.4f}")
        break

loss: 2.0524


In [15]:
bndiff.shape

(128, 200)